In [ ]:
import duckdb
with duckdb.connect("../ads_data.duckdb") as con: 
    df_mart = con.execute("SELECT * FROM mart.mart_kultur_media").df()
df_test = df_mart 

top_employer = df_test.groupby("occupation")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()

top_employer

nr_1 = top_employer[["occupation","description"]].iloc[3]
nr_1.to_csv("for_gemini.txt")

In [4]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
from google import generativeai as oldGenai
from google.generativeai.types import GenerationConfig

# client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
# with duckdb.connect("../../ads_data.duckdb") as con: 
#     df_mart = con.execute("SELECT * FROM mart.mart_kultur_media").df()
# top_employer = df_mart.groupby("occupation")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()

with open("for_gemini.txt", "r", encoding='utf-8') as file: 
    ad_text = file.read() 

max_rows = 100
ad_text_splitted = ad_text.split("\n") #tagit bort alla mellanslag
chunks = [] # lista för textbitar

for i in range(0, len(ad_text_splitted), max_rows): 
    chunk = "\n".join(ad_text_splitted[i:i+max_rows])
    chunks.append(chunk)


response_list = []
for chunk in chunks:
    promt = f"""Du är en rekryterare inom Media, Kultur, Design.
    Plocka ut max 3 av vanligaste kraven och max av 3 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser, samt en sammanfattning av krav och meriter som du skriver ut (om det finns):
    {chunk}

    Output ska vara i detta formatet enbart: 

    {{
        "sammanfattning": ["sammanfattning"]
        "krav": [erfarenhet1, erfarenhet2, ...]
        "antal krav": [summa erfarenhet1..]
        "meriterande": [meriterande1,meriterande2, ... ]
        "antal meriterande": [summa meriterande1..]
        
    }}
    """
    model = oldGenai.GenerativeModel(
        model_name="gemini-2.0-flash",
        generation_config=GenerationConfig(
            temperature=0.0,         
            top_p=1.0,  # Säkerställer att sampling inte påverkas   #verkar inte göra skillnad
            top_k=1     # Tar bort variation)                       #verkar inte göra skillnad    
        )
    )
    response = model.generate_content(promt)
    print(response.text)
    response_list.append(response.text)

```json
{
    "ArkDes": {
        "sammanfattning": [
            "Söker en assistent till bibliotek och forskarservice med fokus på informationsdisk, litteratursökning, och stöd till forskarservice. Vikt läggs vid struktur, noggrannhet, serviceinriktning och intresse för arkitektur och design."
        ],
        "krav": [
            "Relevant akademisk examen (arkitektur, konstvetenskap, biblioteksvetenskap eller likvärdigt)",
            "Dokumenterad erfarenhet av liknande arbetsuppgifter",
            "Arbetslivserfarenhet från bibliotek och museisektorn"
        ],
        "antal krav": [
            1,
            1,
            1
        ],
        "meriterande": [
            "Erfarenhet av att ha jobbat med ArkDes samlingar och/eller bibliotek",
            "Övriga språkkunskaper"
        ],
        "antal meriterande": [
            1,
            1
        ]
    },
    "Södertörns Gymnasium": {
        "sammanfattning": [
            "Söker en skolbibliotekarie/administrat

In [5]:
# import pandas as pd
# import json
# data = response.text
# cleaned = data.strip("```json") #.replace("\n", "").strip().replace("  ", "")
# data = json.loads(cleaned)
# for chunk in data: 
#     response_dict = pd.DataFrame({
#         "sammanfattning": data.get("sammanfattning")
#     })
# response_dict["sammanfattning"] # hämtar ut den sista sammanfattningen

In [10]:
# model2 = oldGenai.GenerativeModel(
#     model_name="gemini-2.0-flash",
#     generation_config=GenerationConfig(
#         temperature=0.0,         
#         top_p=1.0,  # Säkerställer att sampling inte påverkas   #verkar inte göra skillnad
#         top_k=1     # Tar bort variation)                       #verkar inte göra skillnad    
#     )
# )

promt2 = f"""kan du analysera detta och summera till en enda dictionary: 
{response_list}

Output ska vara i detta formatet enbart: 

    {{
        "sammanfattning": ["sammanfattning"]
        "krav": [erfarenhet1, erfarenhet2, ...]
        "antal krav": [summa erfarenhet1..]
        "meriterande": [meriterande1,meriterande2, ... ]
        "antal meriterande": [summa meriterande1..]
        
    }}
    """
response2 = model.generate_content(promt2)
print(response2.text)

```json
{
    "sammanfattning": [
        "Söker en assistent till bibliotek och forskarservice med fokus på informationsdisk, litteratursökning, och stöd till forskarservice. Vikt läggs vid struktur, noggrannhet, serviceinriktning och intresse för arkitektur och design.",
        "Söker en skolbibliotekarie/administratör som ska ansvara för bibliotekets drift, läsfrämjande aktiviteter och ge administrativt stöd till skolan. Vikt läggs vid serviceinriktning, initiativförmåga och god språklig säkerhet.",
        "Söker en driven bibliotekarie med intresse för läsfrämjande gentemot barn och unga, samverkan och nyskapande inom biblioteksverksamhet.",
        "Fokus på barn och ungas fria tid kopplat till läsfrämjande verksamhet. Arbeta läsfrämjande för barn och unga med särskilt fokus på unga i åldern 13-18 år. Inspirera och främja barn och ungas läsning genom medier, aktiviteter och delaktighet - i biblioteket och i samverkan med närområdet.",
        "Tjänsten lämpar sig väl för dig som

In [11]:
import pandas as pd
import json
data = response2.text
cleaned = data.strip("```json") #.replace("\n", "").strip().replace("  ", "")
data = json.loads(cleaned)
sammanfattning = data["sammanfattning"]
sammanfattning

['Söker en assistent till bibliotek och forskarservice med fokus på informationsdisk, litteratursökning, och stöd till forskarservice. Vikt läggs vid struktur, noggrannhet, serviceinriktning och intresse för arkitektur och design.',
 'Söker en skolbibliotekarie/administratör som ska ansvara för bibliotekets drift, läsfrämjande aktiviteter och ge administrativt stöd till skolan. Vikt läggs vid serviceinriktning, initiativförmåga och god språklig säkerhet.',
 'Söker en driven bibliotekarie med intresse för läsfrämjande gentemot barn och unga, samverkan och nyskapande inom biblioteksverksamhet.',
 'Fokus på barn och ungas fria tid kopplat till läsfrämjande verksamhet. Arbeta läsfrämjande för barn och unga med särskilt fokus på unga i åldern 13-18 år. Inspirera och främja barn och ungas läsning genom medier, aktiviteter och delaktighet - i biblioteket och i samverkan med närområdet.',
 'Tjänsten lämpar sig väl för dig som söker en mindre men meningsfull tjänst i en kreativ och pedagogisk m

In [12]:
df_krav = pd.DataFrame({
    "krav": data["krav"], 
    "antal krav": data["antal krav"]
})
df_krav

ValueError: All arrays must be of the same length

In [13]:
df_merit = pd.DataFrame({
    "meriterande": data["meriterande"],
    "antal meriterande": data["antal meriterande"]
})
df_merit

,meriterande,antal meriterande
0,Erfarenhet av att ha jobbat med ArkDes samling...,1
1,Övriga språkkunskaper,1
2,Erfarenhet av arbete i skolmiljö,1
3,Kunskaper i andra språk än engelska,1
4,"Erfarenhet av folkbildning, vuxenutbildning el...",1
5,Erfarenhet av läsfrämjande arbete för barn och...,2
6,Erfarenhet av arbete som skolbibliotekarie,1
7,Erfarenhet av att arbeta med elever som har an...,1
8,Erfarenhet av arbete inom pedagogisk verksamhet,1
9,Ytterligare språk utöver svenska och engelska,1


In [ ]:
with open("embedding.txt", "r") as file:
    text = file.read()
promt3 = f"Kan du säga vad det står här: {text}"
response3 = model.generate_content(promt3)
response3.text

'Det är en lista med 300 flyttal (decimaltal) i vetenskaplig notation (även kallad exponentiell notation).  "e-02" betyder "gånger 10 upphöjt till -2", vilket är samma sak som att dividera med 100.\n\nHär är några exempel på hur man tolkar dem:\n\n*   `-1.374748535454273224e-02`  är  `-0.01374748535454273224`\n*   `2.319343388080596924e-02`  är  `0.02319343388080596924`\n*   `1.300993561744689941e-01` är `0.1300993561744689941`\n\nMed andra ord, flyttalen är representerade som:\n\n`mantissa * 10^(exponent)`\n\nDär mantissan är talet före "e" och exponenten är talet efter "e".\n\nListan verkar vara en serie numeriska värden, men utan ytterligare kontext är det svårt att säga vad de representerar. De kan vara data från en sensor, resultat från en beräkning, eller något annat.\n'

In [14]:
import plotly.express as px
px.bar(df_merit, x="meriterande", y="antal meriterande", title="Meriterande erfarenhet")


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'meriterande=%{x}<br>antal meriterande=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Erfarenhet av att ha jobbat med ArkDes samlingar och/eller bibliotek',
                          'Övriga språkkunskaper', 'Erfarenhet av arbete i skolmiljö',
                          'Kunskaper i andra språk än engelska',
                          'Erfarenhet av folkbildning, vuxenutbildning eller arbete på folkhögskola',
                          'Erfarenhet av läsfrämjande arbete för barn och unga',
                          'Erfarenhet av arbete som skolbibliotekarie',
                          'Erfarenhet av att arbeta med elever som har andra modersmål än svenska',
                          'Erfarenhet av arbete inom pedagogisk verksamhet',
                          'Ytterligare språk utöver svenska och engelska',
                          'Erfarenhet av att ingå i projekt med olika aktörer',
                          'Goda digitala kunskaper', 'Erfarenhet av arbete med barn och ungdomar',
                          'Erfarenhet av läsfrämjande aktiviteter', 'Pedagogisk utbildning',
                          'Erfarenhet av att leda kollegor (arbetsledare, projektledare, teamledare)',
                          'Erfarenhet av att utveckla verksamheter och skapa samarbeten',
                          'God litteraturkännedom',
                          'Kunskaper i programmering (R eller motsvarande)',
                          'Kunskap i verktyg för screening och dataextraktion',
                          'Kunskap om systematiska översikter som forskningsmetod',
                          'Erfarenhet av arbete på medicinskt eller lärosätesbibliotek',
                          'Erfarenhet av att arbeta som bibliotekarie',
                          'Erfarenhet av att arbeta med barn och unga',
                          'Erfarenhet av att utveckla verksamheter, skapa samarbeten och arbeta strategiskt (Rosendalsbiblioteket)',
                          'Arbetslivserfarenhet från högskole- eller universitetsbibliotek (LTH)',
                          'Erfarenhet av att arbeta med e-media (LTH)', 'Goda språkkunskaper',
                          'Erfarenhet inom arbetsområdet', 'Körkort B',
                          'Tidigare arbete som ungdomsbibliotekarie',
                          'Erfarenhet av förändrings- eller utformning av ungdomsverksamhet inom biblioteksområdet',
                          'Andra språkkunskaper utöver svenska och engelska',
                          'Erfarenhet av att leda och utveckla biblioteksverksamhet i en skolmiljö',
                          'Andra behörigheter inom utbildningssektorn',
                          'God förmåga och vana i att uttrycka dig på engelska',
                          'Erfarenhet av biblioteksyrket och att arbeta med barn och unga',
                          'Erfarenhet av skolbiblioteksarbete', 'B-körkort',
                          'Erfarenhet av folkbiblioteksarbete',
                          'Kunskap om säkerhet på nätet och källkritik',
                          'Erfarenhet av att hålla i utbildningar, presentationer eller workshops',
                          'Tidigare erfarenhet av arbete i informationstjänst vid bibliotek',
                          'Erfarenhet av arbete på folkbibliotek',
                          'Arbetat med barn och unga samt med läsfrämjande aktiviteter',
                          'Erfarenhet av arbete på bibliotek',
                          'Erfarenhet av att ha arbetat kreativt med barn och ungdomar'],
                         dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'AQEBAQECAQEBAQEBAgIBAgIBAQEBAQEBAQEBAQEBAQEBAwMDAQEBAwMDAQE